Pythonic data access using climakitae 
--------------------------------------
This notebook showcases helper functions from `climakitae` that enable you to access the AE catalog data **without** using a GUI, while also allowing you to perform spatial subsetting and view the data options in an easy-to-use fashion. These functions could be easily implemented in a python script. <br>

As a reminder, you can access the data using one of the following methods: 
1) the climakitae Selections GUI ([getting_started.ipynb](getting_started.ipynb))
2) using helper functions in the `climakitae` library (this notebook!) 
3) the python library `intake` ([intake_direct_data_download.ipynb](intake_direct_data_download.ipynb))
<br>

This notebook showcases option 2.

In [3]:
from climakitae.core.data_interface import (
    get_data_options, 
    get_subsetting_options, 
    get_data
)

## See all the data options in the catalog 
These options will match those in our AE selections GUI. 

In [4]:
get_data_options()

variable  \
downscaling_method scenario                  timescale                                  
Statistical        Historical Climate        daily          Maximum relative humidity   
                                             daily          Minimum relative humidity   
                                             daily            Specific humidity at 2m   
                                             daily              Precipitation (total)   
                                             daily      Shortwave flux at the surface   
...                                                                               ...   
Dynamical          SSP 5-8.5                 hourly                   NOAA Heat Index   
                                             hourly                   NOAA Heat Index   
                   Historical Reconstruction hourly                   NOAA Heat Index   
                                             hourly                   NOAA Heat Index   
                                             hourly                   NOAA Heat Index   

                                                       resolution  
downscaling_method scenario                  timescale             
Statistical        Historical Climate        daily           3 km  
                                             daily           3 km  
                                             daily           3 km  
                                             daily           3 km  
                                             daily           3 km  
...                                                           ...  
Dynamical          SSP 5-8.5                 hourly         45 km  
                                             hourly          9 km  
                   Historical Reconstruction hourly         45 km  
                                             hourly          9 km  
                                             hourly          3 km  

[1216 rows x 2 columns]

## See the data options for a particular subset of inputs
The `get_data_options` function enables you to input a number of different function arguments, corresponding to the columns in the table above, to subset the table. Inputting no arguments, like we did above, will return the entire range of options.<br><br>First, lets print the function documentation to see the inputs and outputs of the function. If an argument (or "parameter", as listed in the documentation) is listed as "optional", that means you don't have to input anything for that argument. In the case of this function, none of the function arguments are required, so you can simply call the function. 

In [5]:
print(get_data_options.__doc__)

Get data options, in the same format as the Select GUI, given a set of possible inputs.
    Allows the user to access the data using the same language as the GUI, bypassing the sometimes unintuitive naming in the catalog.
    If no function inputs are provided, the function returns the entire AE catalog that is available via the Select GUI

    Parameters
    ----------
    variable: str, optional
        Default to None
    downscaling_method: str, optional
        Default to None
    resolution: str, optional
        Default to None
    timescale: str, optional
        Default to None
    scenario: str or list, optional
        Default to None
    tidy: boolean, optional
        Format the pandas dataframe? This creates a DataFrame with a MultiIndex that makes it easier to parse the options.
        Default to True

    Returns
    -------
    cat_subset: pd.DataFrame
        Catalog options for user-provided inputs
    


If you call the function with **no inputs**, it will simply return the entire catalog! But, let's say you want to see all the data options for statistically downscaled data at 3 km resolution. You'll want to provide inputs for the `downscaling_method` and `resolution` arguments. 

In [6]:
get_data_options(
    downscaling_method = "Statistical", 
    resolution = "3 km"
)

variable  \
downscaling_method scenario           timescale                                         
Statistical        Historical Climate daily                 Maximum relative humidity   
                                      daily                 Minimum relative humidity   
                                      daily                   Specific humidity at 2m   
                                      daily                     Precipitation (total)   
                                      daily             Shortwave flux at the surface   
...                                                                               ...   
                   SSP 5-8.5          monthly           Maximum air temperature at 2m   
                                      monthly           Minimum air temperature at 2m   
                                      monthly      West-East component of Wind at 10m   
                                      monthly    North-South component of Wind at 10m   
                                      monthly                       Wind speed at 10m   

                                                resolution  
downscaling_method scenario           timescale             
Statistical        Historical Climate daily           3 km  
                                      daily           3 km  
                                      daily           3 km  
                                      daily           3 km  
                                      daily           3 km  
...                                                    ...  
                   SSP 5-8.5          monthly         3 km  
                                      monthly         3 km  
                                      monthly         3 km  
                                      monthly         3 km  
                                      monthly         3 km  

[80 rows x 2 columns]

Perhaps you want to see all the data options for daily precipitation. We have several precipitation options in the catalog. You don't need to know the name of these variables; simply use "precipitation" as your input to the function for the `variable` argument.<br><br>The function prefers that your inputs match an actual option in the catalog-- with exact capitalizations and no misspelling-- and will print a warning if your input is not a direct match ("precipitation" is not an option, but "Precipitation (total)" is). The function will then try to make a guess as to what you actually meant. 

In [7]:
get_data_options(
    variable = "precipitation", 
    timescale = "daily"
) 

Input variable='precipitation' is not a valid option.
Closest options: 
- Maximum precipitation
- Precipitation (convective only)
- Precipitation (cumulus portion only)
- Precipitation (grid-scale portion only)
- Precipitation (total)
Outputting data for variable='Maximum precipitation'



variable  \
downscaling_method scenario                  timescale                          
Dynamical          Historical Climate        daily      Maximum precipitation   
                                             daily      Maximum precipitation   
                                             daily      Maximum precipitation   
                   SSP 2-4.5                 daily      Maximum precipitation   
                                             daily      Maximum precipitation   
                   SSP 3-7.0                 daily      Maximum precipitation   
                                             daily      Maximum precipitation   
                                             daily      Maximum precipitation   
                   SSP 5-8.5                 daily      Maximum precipitation   
                                             daily      Maximum precipitation   
                   Historical Reconstruction daily      Maximum precipitation   
                                             daily      Maximum precipitation   
                                             daily      Maximum precipitation   

                                                       resolution  
downscaling_method scenario                  timescale             
Dynamical          Historical Climate        daily          45 km  
                                             daily           9 km  
                                             daily           3 km  
                   SSP 2-4.5                 daily          45 km  
                                             daily           9 km  
                   SSP 3-7.0                 daily          45 km  
                                             daily           9 km  
                                             daily           3 km  
                   SSP 5-8.5                 daily          45 km  
                                             daily           9 km  
                   Historical Reconstruction daily          45 km  
                                             daily           9 km  
                                             daily           3 km

The function can also return a simple pandas DataFrame without the complex MultiIndex. Just set `tidy = False`.

In [8]:
get_data_options(
    variable = "precipitation", 
    timescale = "daily", 
    tidy = False
) 

Input variable='precipitation' is not a valid option.
Closest options: 
- Maximum precipitation
- Precipitation (convective only)
- Precipitation (cumulus portion only)
- Precipitation (grid-scale portion only)
- Precipitation (total)
Outputting data for variable='Maximum precipitation'



,variable,downscaling_method,resolution,timescale,scenario
0,Maximum precipitation,Dynamical,45 km,daily,Historical Climate
1,Maximum precipitation,Dynamical,9 km,daily,Historical Climate
2,Maximum precipitation,Dynamical,3 km,daily,Historical Climate
3,Maximum precipitation,Dynamical,45 km,daily,SSP 2-4.5
4,Maximum precipitation,Dynamical,9 km,daily,SSP 2-4.5
5,Maximum precipitation,Dynamical,45 km,daily,SSP 3-7.0
6,Maximum precipitation,Dynamical,9 km,daily,SSP 3-7.0
7,Maximum precipitation,Dynamical,3 km,daily,SSP 3-7.0
8,Maximum precipitation,Dynamical,45 km,daily,SSP 5-8.5
9,Maximum precipitation,Dynamical,9 km,daily,SSP 5-8.5


## See all the geometry options for spatially subsetting the data during retrieval
These options will match those in our AE selections GUI. This will enable you to retrieve a subset for a specific region.

In [9]:
get_subsetting_options()

geometry
area_subset cached_area                                                                              
states      ID                                      POLYGON ((-117.24269 44.39655, -117.23485 44.3...
            WA                                      MULTIPOLYGON (((-122.57041 48.53786, -122.5686...
            NM                                      POLYGON ((-109.05018 31.48001, -109.04985 31.4...
            CA                                      MULTIPOLYGON (((-118.60443 33.47856, -118.5988...
            CO                                      POLYGON ((-109.06026 38.59933, -109.05955 38.7...
...                                                                                               ...
Stations    San Luis Obispo Airport (KSBP)                                POINT (-120.64406 35.23815)
            Santa Ana John Wayne Airport (KSNA)                           POINT (-117.86746 33.67975)
            Santa Barbara Municipal Airport (KSBA)                         POINT (-119.84249 34.4241)
            Stockton Airport (KSCK)                                       POINT (-121.22637 37.88997)
            Ukiah Municipal Airport (KUKI)                                POINT (-123.20015 39.12781)

[330 rows x 1 columns]

This shows a lot of options! Say you're only interested in California counties. Simply set the argument `area_subset` to "CA counties" to see the all options for counties. The function documentation shows the other options, which also match the values in the column "area_subset" in the table above. 

In [10]:
print(get_subsetting_options.__doc__)

Get all geometry options for spatial subsetting.
    Options match those in selections GUI

    Parameters
    ----------
    area_subset: str
        One of "all", "states", "CA counties", "CA Electricity Demand Forecast Zones", "CA watersheds", "CA Electric Balancing Authority Areas", "CA Electric Load Serving Entities (IOU & POU)", "Stations"
        Defaults to "all", which shows all the geometry options with area_subset as a multiindex

    Returns
    -------
    geom_df: pd.DataFrame
        Geometry options
        Shows only options for one area_subset if input is provided that is not "all"
        i.e. if area_subset = "states", only the options for states will be returned
    


In [11]:
get_subsetting_options(area_subset = "CA counties")

,geometry
cached_area,
Alameda County,"POLYGON ((-122.37312 37.88388, -122.37378 37.8..."
Alpine County,"POLYGON ((-120.07333 38.70109, -120.07332 38.7..."
Amador County,"POLYGON ((-121.02771 38.50011, -121.02771 38.5..."
Butte County,"POLYGON ((-122.06943 39.84053, -122.06886 39.8..."
Calaveras County,"POLYGON ((-120.6318 38.34603, -120.6318 38.345..."
Colusa County,"POLYGON ((-121.91512 38.92535, -121.91491 38.9..."
Contra Costa County,"POLYGON ((-121.69732 37.78244, -121.69084 37.7..."
Del Norte County,"POLYGON ((-124.31611 41.72839, -124.3137 41.72..."
El Dorado County,"POLYGON ((-120.18443 39.03101, -120.18838 39.0..."


You can see all the options for subsetting, and their corresponding geometries, but you don't actually need to use the geometries for subsetting if you use climakitae's data retrieval function-- `get_catalog_data` -- explained in the next section. 

## Retrieve data using the get_data() function
You can easily retrieve data from the Analytics Engine data catalog using climakitae's ```get_data``` function, described below. Additional details for each of the function arguments can be viewed in function docstrings in the next code cell. 

### Required inputs 
This function requires you to input values for the following arguments: 
- variable (required)
- resolution (required)
- timescale (required)

### Location subsetting 
The options for location subsetting can be found using the `get_data_options()` function, as described in the beginning of this notebook. You can also opt to perform an area average by setting `area_average = "Yes"`. The `get_data()` function will default to returning the entire spatial domain, with no area averaging performed. 
- area_subset (optional) 
- cached_area (optional) 
- area_average (optional)

### Additional options
Further modify the data returned using the following arguments.
- downscaling method (optional)
- approach (optional) 
- scenario (optional)
- units (optional)
- time_slice (optional)
- warming_level (optional)
- warming_level_window (optional)
- warming_level_months (optional)

In [12]:
# See additional details about the function arguments by printing the docstring
print(get_data.__doc__)

Retrieve formatted data from the Analytics Engine data catalog using a simple function.
    Contrasts with DataParameters().retrieve(), which retrieves data from the user inputs in climakitaegui's selections GUI.

    Parameters
    ----------
    variable: str
        String name of climate variable
    resolution: str, one of ["3 km", "9 km", "45 km"]
        Resolution of data in kilometers
    timescale: str, one of ["hourly", "daily", "monthly"]
        Temporal frequency of dataset
    downscaling_method: str, one of ["Dynamical", "Statistical", "Dynamical+Statistical"], optional
        Downscaling method of the data:
        WRF ("Dynamical"), LOCA2 ("Statistical"), or both "Dynamical+Statistical"
        Default to "Dynamical"
    data_type: str, one of ["Gridded", "Stations"], optional
        Whether to choose gridded data or weather station data
        Default to "Gridded"
    approach: one of ["Time", "Warming Level"], optional
        Default to "Time"
    scenario: str 

### Example 1: Time-based approach
Retrieve gridded data using a time-based approach. ```approach``` is an optional function argument, but the default is to use a time-based approach, so you don't actually need to set this argument. 

#### Example 1a
First, let's retrieve 3 kilometer resolution statistically downscaled historical data at a monthly timestep. 

In [13]:
get_data(
    variable = "Precipitation (total)", 
    downscaling_method = "Statistical", 
    resolution = "3 km", 
    timescale = "monthly", 
    scenario = "Historical Climate"
    # approach = "Time" # Optional because "Time" is the function default 
)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!! Returned data array is huge. Operations could take 10x to infinity longer than 1GB of data !!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



<xarray.DataArray np.str_('Precipitation (total)') (scenario: 1,
                                                    simulation: 70, time: 780,
                                                    lat: 495, lon: 559)> Size: 121GB
dask.array<broadcast_to, shape=(1, 70, 780, 495, 559), dtype=float64, chunksize=(1, 1, 308, 310, 351), chunktype=numpy.ndarray>
Coordinates:
  * lat          (lat) float32 2kB 29.58 29.61 29.64 29.67 ... 44.95 44.98 45.02
  * lon          (lon) float32 2kB -128.4 -128.4 -128.4 ... -111.0 -111.0 -111.0
  * time         (time) datetime64[ns] 6kB 1950-01-01 1950-02-01 ... 2014-12-01
  * simulation   (simulation) <U30 8kB 'LOCA2_EC-Earth3_r1i1p1f1' ... 'LOCA2_...
  * scenario     (scenario) <U18 72B 'Historical Climate'
    spatial_ref  int64 8B 0
Attributes:
    variable_id:           pr
    extended_description:  Total precipitation. Computed by summing total gri...
    units:                 kg m-2 s-1
    data_type:             Gridded
    resolution:            3 km
    frequency:             monthly
    location_subset:       ['entire domain']
    approach:              Time
    downscaling_method:    Statistical
    institution:           UCSD

#### Example 1b
Now say you're only interested in this data for San Bernadino County, and you want to compute an area average over the entire county. 

In [14]:
get_data(
    variable = "Precipitation (total)", 
    downscaling_method = "Statistical", 
    resolution = "3 km", 
    timescale = "monthly", 
    scenario = "Historical Climate",
    
    # Modify location settings
    cached_area = "San Bernardino County", 
    area_average = "Yes"
)

<xarray.DataArray np.str_('Precipitation (total)') (scenario: 1,
                                                    simulation: 70, time: 780)> Size: 437kB
dask.array<broadcast_to, shape=(1, 70, 780), dtype=float64, chunksize=(1, 1, 308), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 6kB 1950-01-01 1950-02-01 ... 2014-12-01
  * simulation   (simulation) <U30 8kB 'LOCA2_EC-Earth3_r1i1p1f1' ... 'LOCA2_...
  * scenario     (scenario) <U18 72B 'Historical Climate'
    spatial_ref  int64 8B 0
Attributes:
    variable_id:           pr
    extended_description:  Total precipitation. Computed by summing total gri...
    units:                 kg m-2 s-1
    data_type:             Gridded
    resolution:            3 km
    frequency:             monthly
    location_subset:       ['San Bernardino County']
    approach:              Time
    downscaling_method:    Statistical
    institution:           UCSD

#### Example 1c 
Perhaps next you want to get dynamically downscaled (i.e. WRF) precipitation data instead. First, you might want to check what options you have for scenario, timescale, and resolution using the ```get_data_options()``` function. 

In [15]:
get_data_options(
    variable = "Precipitation (total)", 
    downscaling_method = "Dynamical"
) 

variable  \
downscaling_method scenario                  timescale                          
Dynamical          Historical Climate        hourly     Precipitation (total)   
                                             hourly     Precipitation (total)   
                                             hourly     Precipitation (total)   
                   SSP 3-7.0                 hourly     Precipitation (total)   
                                             hourly     Precipitation (total)   
                                             hourly     Precipitation (total)   
                   Historical Climate        daily      Precipitation (total)   
                                             daily      Precipitation (total)   
                                             daily      Precipitation (total)   
                                             monthly    Precipitation (total)   
                                             monthly    Precipitation (total)   
                                             monthly    Precipitation (total)   
                   SSP 2-4.5                 hourly     Precipitation (total)   
                                             hourly     Precipitation (total)   
                                             daily      Precipitation (total)   
                                             daily      Precipitation (total)   
                                             monthly    Precipitation (total)   
                                             monthly    Precipitation (total)   
                   SSP 3-7.0                 daily      Precipitation (total)   
                                             daily      Precipitation (total)   
                                             daily      Precipitation (total)   
                                             monthly    Precipitation (total)   
                                             monthly    Precipitation (total)   
                                             monthly    Precipitation (total)   
                   SSP 5-8.5                 hourly     Precipitation (total)   
                                             hourly     Precipitation (total)   
                                             daily      Precipitation (total)   
                                             daily      Precipitation (total)   
                                             monthly    Precipitation (total)   
                                             monthly    Precipitation (total)   
                   Historical Reconstruction hourly     Precipitation (total)   
                                             hourly     Precipitation (total)   
                                             hourly     Precipitation (total)   
                                             daily      Precipitation (total)   
                                             daily      Precipitation (total)   
                                             daily      Precipitation (total)   
                                             monthly    Precipitation (total)   
                                             monthly    Precipitation (total)   
                                             monthly    Precipitation (total)   

                                                       resolution  
downscaling_method scenario                  timescale             
Dynamical          Historical Climate        hourly         45 km  
                                             hourly          9 km  
                                             hourly          3 km  
                   SSP 3-7.0                 hourly         45 km  
                                             hourly          9 km  
                                             hourly          3 km  
                   Historical Climate        daily          45 km  
                                             daily           9 km  
                                             daily           3 km  

Next, let's retrieve both the future and historical dynamically downscaled data. "Historical Climate" is the correct historical data option here; "Historical Reconstruction" data is from ERA5 (a climate reanalysis product, rather than a climate model), and cannot be retrieved with future data in the same function call. <br><br>You can set the ```scenario``` argument to retrieve the shared socioeconomic pathway data (future projections) appended to the historical data. You can also set your desired time period using the ```time_slice``` argument. 

In [16]:
get_data(
    variable = "Precipitation (total)", 
    downscaling_method = "Dynamical", 
    resolution = "45 km", 
    timescale = "monthly", 
    cached_area = "San Bernardino County", 
    
    # Modify time-based settings 
    time_slice = (2000,2050),
    scenario = [
        "Historical Climate", 
        "SSP 3-7.0", 
        "SSP 2-4.5",
        "SSP 5-8.5"
    ]
) 

WARNING
-------
You have retrieved data for more than one SSP, but not all ensemble members for each GCM are available for all SSPs.

As a result, some scenario and simulation combinations may contain NaN values.

If you want to remove these empty simulations, it is recommended to first subset the data object by each individual scenario and then dropping NaN values.


<xarray.DataArray np.str_('Precipitation (total)') (scenario: 3, simulation: 8,
                                                    time: 612, y: 7, x: 7)> Size: 3MB
dask.array<concatenate, shape=(3, 8, 612, 7, 7), dtype=float32, chunksize=(1, 1, 436, 7, 7), chunktype=numpy.ndarray>
Coordinates:
  * time               (time) datetime64[ns] 5kB 2000-01-01 ... 2050-12-01
  * x                  (x) float64 56B -4.08e+06 -4.035e+06 ... -3.81e+06
  * y                  (y) float64 56B 6.739e+05 7.189e+05 ... 9.439e+05
  * simulation         (simulation) <U26 832B 'WRF_CESM2_r11i1p1f1' ... 'WRF_...
    lakemask           (y, x) float32 196B 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    landmask           (y, x) float32 196B 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    lat                (y, x) float32 196B 32.91 33.14 33.36 ... 36.1 36.33
    lon                (y, x) float32 196B -116.8 -116.4 ... -116.4 -116.0
    Lambert_Conformal  int64 8B 0
  * scenario           (scenario) <U22 264B 'Historical + SSP 5-8.5' ... 'His...
Attributes:
    variable_id:           prec
    extended_description:  Total precipitation. Computed by summing total gri...
    units:                 mm
    data_type:             Gridded
    resolution:            45 km
    frequency:             monthly
    location_subset:       ['San Bernardino County']
    approach:              Time
    downscaling_method:    Dynamical
    institution:           UCLA
    grid_mapping:          Lambert_Conformal

### Example 2: Warming levels approach 
By default, the function uses a time-based approach. To use a warming levels approach, set the argument ```approach = "Warming Level"```. 

#### Example 2a
Retrieve the same data as example 1c, using a warming levels approach instead of a time-based approach. Note that the ```scenario``` and ```time_slice``` arguments are invalid for a warming levels approach; if provided, they will be ignored by the function. 

In [17]:
get_data(
    variable = "Precipitation (total)", 
    downscaling_method = "Dynamical", 
    resolution = "45 km", 
    timescale = "monthly", 
    cached_area = "San Bernardino County", 
    
    # Modify your approach 
    approach = "Warming Level",
)

<xarray.DataArray np.str_('Precipitation (total)') (warming_level: 1,
                                                    time_delta: 360, y: 7,
                                                    x: 7, simulation: 10)> Size: 706kB
dask.array<getitem, shape=(1, 360, 7, 7, 10), dtype=float32, chunksize=(1, 304, 7, 7, 1), chunktype=numpy.ndarray>
Coordinates:
  * warming_level      (warming_level) float64 8B 2.0
  * x                  (x) float64 56B -4.08e+06 -4.035e+06 ... -3.81e+06
  * y                  (y) float64 56B 6.739e+05 7.189e+05 ... 9.439e+05
    lakemask           (y, x) float32 196B 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    landmask           (y, x) float32 196B 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    lat                (y, x) float32 196B 32.91 33.14 33.36 ... 36.1 36.33
    lon                (y, x) float32 196B -116.8 -116.4 ... -116.4 -116.0
    Lambert_Conformal  int64 8B 0
  * time_delta         (time_delta) float64 3kB -180.0 -179.0 ... 178.0 179.0
    centered_year      (simulation) int64 80B 2039 2043 2035 ... 2053 2045 2040
  * simulation         (simulation) <U44 2kB 'WRF_CESM2_r11i1p1f1_historical+...
Attributes:
    variable_id:           prec
    extended_description:  Total precipitation. Computed by summing total gri...
    units:                 mm
    data_type:             Gridded
    resolution:            45 km
    frequency:             monthly
    location_subset:       ['San Bernardino County']
    approach:              Warming Level
    downscaling_method:    Dynamical
    warming_level_window:  +/- 15 years from centered year
    institution:           UCLA
    grid_mapping:          Lambert_Conformal

#### Example 2b
The ```get_data()``` function uses a default warming levels window of +/- 15 years, resulting in a 30 year period. Lets modify that by setting ```warming_level_window = 10``` to retrieve a 20 year window.<br><br>We can also modify the warming levels computed to include additional warming levels beyond the default. Let's select a few more by setting ```warming_level = [2.5, 3.0, 4.0]```. 

In [18]:
get_data(
    variable = "Precipitation (total)", 
    downscaling_method = "Dynamical", 
    resolution = "45 km", 
    timescale = "monthly", 
    cached_area = "San Bernardino County", 
    approach = "Warming Level",
    
    # Modify warming level settings 
    warming_level_window = 10, 
    warming_level = [2.5, 3.0, 4.0]
)

WARNING FOR WARMING LEVELS APPROACH
-----------------------------------
There may be NaNs in your data for certain simulation/warming level combinations if the warming level is not reached for that particular simulation before the year 2100. 

This does not mean you have missing data, but rather a feature of how the data is combined in retrieval to return a single data object. 

If you want to remove these empty simulations, it is recommended to first subset the data object by each individual warming level and then dropping NaN values.


<xarray.DataArray np.str_('Precipitation (total)') (warming_level: 3,
                                                    time_delta: 240, y: 7,
                                                    x: 7, simulation: 10)> Size: 1MB
dask.array<concatenate, shape=(3, 240, 7, 7, 10), dtype=float32, chunksize=(1, 240, 7, 7, 1), chunktype=numpy.ndarray>
Coordinates:
  * x                  (x) float64 56B -4.08e+06 -4.035e+06 ... -3.81e+06
  * y                  (y) float64 56B 6.739e+05 7.189e+05 ... 9.439e+05
  * time_delta         (time_delta) float64 2kB -120.0 -119.0 ... 118.0 119.0
  * warming_level      (warming_level) float64 24B 2.5 3.0 4.0
    lakemask           (y, x) float32 196B 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    landmask           (y, x) float32 196B 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
    lat                (y, x) float32 196B 32.91 33.14 33.36 ... 36.1 36.33
    lon                (y, x) float32 196B -116.8 -116.4 ... -116.4 -116.0
    Lambert_Conformal  int64 8B 0
    centered_year      (warming_level, simulation) float64 240B 2.059e+03 ......
  * simulation         (simulation) <U44 2kB 'WRF_CESM2_r11i1p1f1_historical+...
Attributes:
    variable_id:           prec
    extended_description:  Total precipitation. Computed by summing total gri...
    units:                 mm
    data_type:             Gridded
    resolution:            45 km
    frequency:             monthly
    location_subset:       ['San Bernardino County']
    approach:              Warming Level
    downscaling_method:    Dynamical
    warming_level_window:  +/- 10 years from centered year
    institution:           UCLA
    grid_mapping:          Lambert_Conformal

### Example 3: Weather station data 
By default, the function retrieves non-bias-corrected gridded data, but you can also retrieve dynamically downscaled climate data that has been bias-corrected using historical weather station data. This data is retrieved at the unique grid cell(s) corresponding to the selected weather station(s). This data can be retrieved using the `data_type` and `stations` arguments. If you don't set the `stations` argument, the function will return all available weather stations-- a hefty data retrieval that takes a while to run and is therefore is not recommended. 

```
data_type = "Stations" # Return bias-corrected gridded data at a station(s) of interest 
data_type = "Gridded" # Return gridded data (function default) 
```

As of now, you can only retrieve hourly data for the variable "Air Temperature at 2m". You can also choose the resolution of the gridded data used in bias correction by setting the `resolution` argument to either "3 km" or "9 km".

#### Example 3a

In [19]:
get_data(
    variable = "Air Temperature at 2m", # Required argument
    resolution = "9 km", # Required argument. Options: "9 km" or "3 km" 
    timescale = "hourly", # Required argument
    data_type = "Stations", # Required argument
    stations = "San Diego Lindbergh Field (KSAN)" # Optional argument. If no input, all weather stations are retrieved 
)

/home/jovyan/src/climakitae/climakitae/core/data_load.py:1577: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  da_adj["time"] = da_adj.indexes["time"].to_datetimeindex()


<xarray.Dataset> Size: 21MB
Dimensions:                           (scenario: 1, simulation: 8, time: 297840)
Coordinates:
  * scenario                          (scenario) <U18 72B 'Historical Climate'
  * simulation                        (simulation) <U26 832B 'WRF_CESM2_r11i1...
  * time                              (time) datetime64[ns] 2MB 1980-09-01 .....
Data variables:
    San Diego Lindbergh Field (KSAN)  (time, scenario, simulation) float64 19MB dask.array<chunksize=(297840, 1, 1), meta=np.ndarray>

To see all the available weather station options, you can use the `get_subsetting_options()` function detailed at the top of this notebook. Simply set the `area_subset` function argument to `"Stations"`. 

In [20]:
get_subsetting_options(area_subset="Stations") 

,geometry
cached_area,
Arcata Eureka Airport (KACV),POINT (-124.10479 40.97844)
Bakersfield Meadows Field (KBFL),POINT (-119.05524 35.43424)
Blythe Asos (KBLH),POINT (-114.71451 33.61876)
Burbank-Glendale-Pasadena Airport (KBUR),POINT (-118.36543 34.19966)
Desert Resorts Regional Airport (KTRM),POINT (-116.16412 33.63166)
Downtown Los Angeles USC Campus (KCQT),POINT (-118.291 34.024)
Fresno Yosemite International Airport (KFAT),POINT (-119.72016 36.77999)
Gillespie Field Airport (KSEE),POINT (-116.9725 32.82611)
Imperial County Airport (KIPL),POINT (-115.57656 32.83464)


#### Example 3b
To demonstrate the flexibility of this function, let's make a few changes to the function argument in the code below: 
1) Retrieve more than one weather station. 
2) Change the resolution of the data used for bias-correction
3) Change the variable units (function default to degrees Kelvin, the native unit of the raw data)
4) Change the timescale to retrieve a 5 year period (function defaults to the entire historical record: 1980-2014)

Note that this function will take more time to run since we're retrieving more than one station. 

In [ ]:
get_data(
    variable = "Air Temperature at 2m", 
    resolution = "3 km",
    timescale = "hourly",
    data_type = "Stations",
    stations = [
        "San Francisco International Airport (KSFO)", 
        "Oakland Metro International Airport (KOAK)", 
    ],
    units = "degF", 
    time_slice = (2000,2005) 
)

#### Example 3c 
You can also retrieve data for the future period by setting the `scenario` argument to one or more SSPs. The code below will retrieve data for 2000-2035 for the Sacramento Executive Airport (KSAC) station under Shared Socioeconomic Pathway 3-7.0. Since we are retrieving data in both the historical and future period, we need to set the scenario to `['Historical Climate', 'SSP 3-7.0']`.

In [ ]:
get_data(
    variable = "Air Temperature at 2m", 
    resolution = "9 km",
    timescale = "hourly",
    data_type = "Stations",
    stations = "Sacramento Executive Airport (KSAC)",
    units = "degF", 
    time_slice = (2000,2035),
    scenario = ["Historical Climate", "SSP 3-7.0"]
)